Pandas importieren und Excel Datei in DataFrame einlesen

In [2]:
import pandas as pd

df = pd.read_excel("../Lets Meet DB Dump.xlsx")
print(df.head())

    Nachname, Vorname                            Straße Nr, PLZ Ort  \
0     Forster, Martin            Minslebener Str. 0, 46286, Dorsten   
1  Elina, Tsanaklidou               Gartenweg 13, 69126, Heidelberg   
2       Vildan, şahin  Heinrich-Heine-Straße 99c, 17489, Greifswald   
3      Bäumker, Ellen                 Weidenring 39, 81539, München   
4  Bahadır, Bekiroğlu   Eichendorffstraße 20, 79379, Müllheim Baden   

         Telefon  \
0     02372 8020   
1  06221 / 98689   
2  03834 / 22951   
3  0162 / 249788   
4  (07631) 67955   

  Hobby1 %Prio1%; Hobby2 %Prio2%; Hobby3 %Prio3%; Hobby4 %Prio4%; Hobby5 %Prio5%;  \
0  Fremdsprachenkenntnisse erweitern %78%; Im Was...                                
1  Jemanden massieren %57%; Mir die Probleme von ...                                
2  In der Stadt herumbummeln %94%; Mit Freunden z...                                
3  An einer Gruppenreise teilnehmen %97%; Seine f...                                
4  Ins Grüne fahren %23%; H

Saubere Namensvergaben der Spalten

In [3]:
rename_map = {
    "Nachname, Vorname": "name",
    "Straße Nr, PLZ Ort": "adresse",
    "Telefon": "telefon",
    "Hobby1 %Prio1%; Hobby2 %Prio2%; Hobby3 %Prio3%; Hobby4 %Prio4%; Hobby5 %Prio5%;": "hobbys",
    "E-Mail": "email",
    "Geschlecht (m/w/nonbinary)": "geschlecht",
    "Interessiert an": "interessiert_an",
    "Geburtsdatum": "geburtsdatum"
}

df = df.rename(columns=rename_map)


Bereinigung der Zellen

In [4]:
# 1.1   Führende und folgende Leerzeichen werden mit strip() entfernt. Außerdem werden leere Felder mit NA gefüllt durch pd.NA.
for c in df.columns:
    if df[c].dtype == "object":
        df[c] = df[c].str.strip()
df = df.replace({"": pd.NA})

# 1.2   E-Mail Adressen werden durch .str.lower() einheitlich klein geschrieben und mit str.strip() wird sicher gegangen, dass alle Leerzeichen am Anfang und Ende entfernt sind
#       In der Spalte Telefon werden mit str.replace(r"[^\d+]", "", regex=True) alle Zeichen, außer die Zahlen 0-9 (\d) und das Plus entfernt. r vor der eckigen Klammer sorgt dafür, dass Python die Backslashes ignoriert. regex=True bedeutet, dass der Ausdruck als Regex-Ausdruck behandelt werden soll, was eine Sprache für Muster in Texten ist.
df["email"] = df["email"].str.lower().str.strip()
df["telefon"] = df["telefon"].str.replace(r"[^\d+]", "", regex=True)

# 1.3 Geschlecht normalisieren (m/w/nb → Männlich/Weiblich/Nichtbinär)
df["geschlecht_norm"] = (
    df["geschlecht"].str.lower()
    .replace({"m":"männlich","w":"weiblich", "nb":"nichtbinär"})
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
    .str.capitalize()
)

# 1.4 Namen/Adresse bei Bedarf trennen (nur falls nötig)
# Name -> Vorname/Nachname (einfaches Pattern)
if "vorname" not in df and "nachname" not in df and "name" in df:
    parts = df["name"].fillna("").str.extract(r"^(?P<nachname>[^,]+),\s*(?P<vorname>.+)$")
    df["nachname"], df["vorname"] = parts["nachname"], parts["vorname"]


# Adresse -> Straße/Hausnummer/PLZ/Ort (Format: "Straße Hausnummer, PLZ, Ort")
if "adresse" in df:
    addr = (
        df["adresse"]
        .fillna("")
        .str.replace(r"\s+", " ", regex=True)  # Mehrfachspaces glätten
        .str.strip()
    )

    # Ein Regex für: Straße(namen) + Hausnummer, Komma, PLZ, Komma, Ort
    m = addr.str.extract(
        r"^(?P<straße>.+?)\s+(?P<hausnummer>\S+),\s*(?P<plz>\d{5}),\s*(?P<ort>.+?)\s*$"
    )

    df["straße"]     = m["straße"]
    df["hausnummer"] = m["hausnummer"]
    df["plz"]        = m["plz"]
    df["ort"]        = m["ort"]

# 1.5 Hobbys: Kommaliste in Array
df["hobbies_list"] = df["hobbys"].fillna("").str.split(r"\s*,\s*", regex=True)



Hobbys ohne Mehrfachnennung in eigenen Dataframe überführen

In [12]:

# 1) Spalte "hobbys" in einzelne Einträge aufsplitten
df_hobby_entries = (
    df[["email", "hobbys"]]
    .assign(entry = df["hobbys"].fillna("").str.split(r"\s*;\s*"))  # an Semikolons trennen
    .explode("entry")                                              # jede Hobby/Prio-Kombi eigene Zeile
)

# 2) Leere rausfiltern
df_hobby_entries = df_hobby_entries[df_hobby_entries["entry"].notna()]
df_hobby_entries["entry"] = df_hobby_entries["entry"].str.strip()

# 3) Hobbyname und Prio per Regex extrahieren
# Muster: "Text %Zahl%"
pattern = r"^(?P<hobby>.+?)\s*%\s*(?P<praef>-?\d{1,3})\s*%$"
parts = df_hobby_entries["entry"].str.extract(pattern)

df_hobby_entries["hobby"] = parts["hobby"].str.strip().str.title()
df_hobby_entries["praeferenz"] = pd.to_numeric(parts["praef"], errors="coerce").clip(-100, 100)

# 4) DataFrame nur mit allen einzigartigen Hobbys
df_hobbies = df_hobby_entries["hobby"].dropna().drop_duplicates().reset_index(drop=True)
df_hobbies = df_hobbies.to_frame(name="Hobby")
df_hobbies["ID"] = df_hobbies.index + 1


df_hobbies


,Hobby,ID
0,Fremdsprachenkenntnisse Erweitern,1
1,Im Wasser Waten,2
2,Schwierige Probleme Klären,3
3,Morgens Früh Aufstehen,4
4,Jemanden Massieren,5
...,...,...
215,"Laufen, Joggen, Nordic Walking",216
216,Bowling Spielen,217
217,Radfahren,218
218,"Freiwillige Arbeit Tun, An Gemeinnützigen Proj...",219


In [ ]:
hobby_counts = df_hobby_entries["hobby"].value_counts()

print(hobby_counts)

hobby
Etwas Entwerfen Oder Zeichnen                                 80
Romane, Erzählungen, Theaterstücke Oder Gedichte Schreiben    80
Mit Bekannten Telefonieren                                    80
Etwas Lernen Und Mich Fortbilden                              78
Auf Mich Zukommende Situationen In Gedanken Bewältigen        76
                                                              ..
Bowling Spielen                                                4
Studieren                                                      2
Tischtennis Spielen                                            2
In Einer Musikgruppe Mitspielen                                2
Laufen, Joggen, Nordic Walking                                 1
Name: count, Length: 220, dtype: int64


In [14]:
hobby_counts_df = (
    df_hobby_entries.groupby("hobby")["email"]
    .count()
    .reset_index()
    .rename(columns={"email": "anzahl_nutzer"})
    .sort_values("anzahl_nutzer", ascending=False)
)

print(hobby_counts_df)


                                                 hobby  anzahl_nutzer
57                       Etwas Entwerfen Oder Zeichnen             80
135                         Mit Bekannten Telefonieren             80
165  Romane, Erzählungen, Theaterstücke Oder Gedich...             80
60                    Etwas Lernen Und Mich Fortbilden             78
12   Auf Mich Zukommende Situationen In Gedanken Be...             76
..                                                 ...            ...
167                        Schauspielerisch Tätig Sein              4
187                                          Studieren              2
98                     In Einer Musikgruppe Mitspielen              2
192                                Tischtennis Spielen              2
117                     Laufen, Joggen, Nordic Walking              1

[220 rows x 2 columns]
